In [1]:
from linear_sample_3c1 import *
import time
from collections import namedtuple
from read_images import *#不带模糊
#from read_images_flur import *#带模糊，输出左图，右图和模糊
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

/home/ubuntu/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ubuntu/anaconda3/envs/py36/lib/python3.6/site-

In [2]:
model_params = namedtuple("parameters",
                          'data_size,'
                           'mini_batch_size,'
                           'learning_rate,'
                           'total_epoch_num,'
                           'outputdir,'
                           'height,'
                           'width,'
                           'channel,'
                          'cutTop,'
                          'cutBottom,'
                          'cutLeft,'
                          'cutRight,'
                          'cPointRow,'
                          'cPointCol')


In [3]:
def train_feature(params, feature_in, left_ims, right_ims, tps_base, tps_matrix, linear_interpolator, mode, loss_norm_mode=True):
    print_str = 'train_tps_Step：{:4} | Reonstruction loss is {:4} | Norm loss is {:4} | Total loss is {:4}' \
                '  | feature_var_mean is{:4}'
    # with tf.Graph().as_default(), tf.device('/gpu: 0'):
    max_step = np.int32(params.total_epoch_num)
    learning_rate_init = np.float32(params.learning_rate)

    optimize_op = tf.train.AdamOptimizer(learning_rate_init)
    
    
    with tf.variable_scope(tf.get_variable_scope()):
        tps_base = tf.constant(tps_base, dtype=tf.float32)
        left = tf.constant(left_ims, dtype=tf.float32)
        right = tf.constant(right_ims, dtype=tf.float32)
        if mode == 'TPS':
            feature_input = tf.Variable(feature_in, dtype=tf.float32,name='contr_val', trainable=False)
            tps_weight = tf.Variable(tps_matrix, dtype=tf.float32, name='tps_val')
        elif mode == 'FEATURE':
            feature_input = tf.Variable(feature_in, dtype=tf.float32, name='contr_val')
            tps_weight = tf.Variable(tps_matrix, dtype=tf.float32, name='tps_val', trainable=False)
        else:
            feature_input = tf.Variable(feature_in, dtype=tf.float32)
            tps_weight = tf.Variable(tps_matrix, dtype=tf.float32)
        
        compensateI = tf.Variable(4.3, dtype=tf.float32, name='contr_val')###亮度补偿
        #compensateI = tf.Variable(-2.0, dtype=tf.float32, name='contr_val')###亮度补偿
        
        
        
        disp = decoder_forward1(feature_input, tps_weight, linear_interpolator.sz_params)
        #train_dd__ = np.load('groundtruth_z/disp_batch_invivo_d.npy')[0:100]
        #disp = tf.Variable(train_dd__, dtype=tf.float32)
        
        
        
        right_est = linear_interpolator.interpolate(left, disp)
        #loss_rec = compute_rec_loss(right_est, right, linear_interpolator.sz_params)
        loss_rec,compa_sum,loss_rec_sum = compute_rec_loss(right_est, right, compensateI, linear_interpolator.sz_params, 180.0)##使用膨胀做部分平滑
        #loss_rec = compute_rec_loss3(right_est, right, left, linear_interpolator.sz_params)
        loss_wt_norm = tf.reduce_sum(tf.square(tps_weight - tps_base))
        
        #disp_i1 = tf.slice(disp, [0,  params.cutTop, params.cutLeft, 0], [-1, 256, 255, -1],
        #                 name='r_clip')
        #disp_i2 = tf.slice(disp, [0,  params.cutTop, params.cutLeft+1, 0], [-1, 256, 255, -1],
        #                 name='r_clip')
        #disp_i3 = tf.slice(disp, [0,  params.cutTop, params.cutLeft, 0], [-1, 255, 256, -1],
        #                 name='r_clip')
        #disp_i4 = tf.slice(disp, [0,  params.cutTop+1, params.cutLeft, 0], [-1, 255, 256, -1],
        #                 name='r_clip')
        
        #loss_wt_norm = tf.multiply(0.0015, tf.reduce_sum(tf.square(disp_i2 - disp_i1)), name='punishment')##横向
        #loss_wt_norm2 = tf.multiply(0.002, tf.reduce_sum(tf.square(disp_i4 - disp_i3)), name='punishment2')##纵向
        #loss = tf.add(loss_rec, loss_wt_norm, name='Total_loss')
        #loss = tf.add(loss, loss_wt_norm2, name='Total_loss')
        loss = tf.add(loss_rec, loss_wt_norm, name='Total_loss')
        
        if mode == 'FEATURE' or loss_norm_mode == False:
            #train_op = optimize_op.minimize(loss,var_list = [feature_input, compensateI])#可以指定哪些参数进行迭代
            train_op = optimize_op.minimize(loss,var_list = feature_input)
            pass
        else:
            #train_op = optimize_op.minimize(loss,var_list = [feature_input, compensateI])
            train_op = optimize_op.minimize(loss,var_list = feature_input)
            pass
    
    config = tf.ConfigProto(allow_soft_placement=True)
    print(1)
    with tf.Session(config=config) as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.local_variables_initializer())
        print(1.1)
        start_time = time.time()
        loss_seq = np.zeros([max_step, 3])
        loss_rec_temp = 0.
        print(1.2)
        
        feature_before = feature_in
        feature_val = 0.
        tps_before = tps_matrix
        tps_val = 0.
        disp_val = np.zeros([params.data_size, params.height, params.width, 1])
        est_right_val = np.zeros([params.data_size, params.height, params.width, params.channel])
        step = 0
        print(2)
        res_loss = [[],[]]
        for step in range(0, max_step):
            _, feature_val, tps_val, loss_rec_val, loss_wt_norm_val, compa_sum_val,loss_rec_sum_val, loss_val, disp_val, est_right_val = sess.run(
                [train_op, feature_input, tps_weight, loss_rec, loss_wt_norm, compa_sum,loss_rec_sum, loss, disp, right_est]
            )


            loss_seq[step, :] = loss_rec_val, loss_wt_norm_val, loss_val
            res_loss[0].append(loss_rec_sum_val*params.data_size)##记录损失
            res_loss[1].append(compa_sum_val)##记录损失
            print(loss_rec_sum_val*params.data_size/compa_sum_val)
            if 0 == step % 10 or step + 1 == max_step:
                print('亮度补偿', sess.run(compensateI))
                #res_loss[0].append(loss_rec_val*params.data_size)##记录损失
                #res_loss[1].append(compa_sum_val)##记录损失
                tps_var_mean = np.mean(tps_val - tps_before)
                feature_var_mean = np.mean(feature_val - feature_before)
                

                print(print_str.format(step, loss_rec_val, loss_wt_norm_val, loss_val, feature_var_mean))#当前组的loss信息
                tps_before = tps_val
                feature_before = feature_val

                loss_rec_var = np.abs(loss_rec_val - loss_rec_temp)
                loss_rec_temp = loss_rec_val
                #if loss_rec_var < 1e-2 or step>=100:#######考虑使用loss，比较一下和重建误差的区别
                if step>=100:
                    break
                #loss_var_chazhi = np.abs(loss_val - loss_rec_temp)
                #loss_rec_temp = loss_val
                #if  loss_var_chazhi < 1e-2 or step>=3000:#######考虑使用loss，比较一下和重建误差的区别
                #    break
                
                
        feature = feature_val
        tps_mat = tps_val
        print('time spent {:8} '.format(time.time() - start_time))
        distance = np.mean(np.square(feature - feature_in))
        print('初始与终值的距离为:'+str(distance))
        
        return feature, tps_mat, loss_seq[0:step + 1, :], disp_val, est_right_val, distance, loss_rec_val, res_loss


def train_z(params, feature_in, tps_base0, left_ims, right_ims, result_path,iii):
    with tf.Graph().as_default(), tf.device('/gpu: 0'):
        linear_interpolator = LinearInterpolator(params)  # initialize linear interpolator
        tps_matrix = tps_base0
        loss_seq = np.empty(shape=[0, 3], dtype=np.float32)
        mode = 'FEATURE'
        print('--------------------FEATURE---------------------')
        feature_in, tps_matrix, loss_seq1, disp, est_right, distance, loss_rec_val, res_loss= train_feature(params, feature_in, left_ims, right_ims, tps_base0,
                                                                           tps_matrix, linear_interpolator, mode)
        ##loss_seq = np.concatenate([loss_seq, loss_seq1], axis=0)
        
        #np.save(os.path.join(result_path, 'disp_batch_invivo20'+str(iii)+'.npy'), disp)
        #np.save(os.path.join(result_path, 'feature_trained'+str(iii)+'.npy'), feature_in)

        return feature_in, res_loss, disp

In [4]:
#通过83.2训练出来的结果 4*5, 训练集
if __name__ == '__main__':
    params = model_params(data_size=1,
                              mini_batch_size=1,
                              learning_rate=3e-1,
                              total_epoch_num=np.int32(10000),
                              outputdir=r'output',
                              height=288,
                              width=360,
                              channel=3,
                              #cutTop=54,
                              #cutBottom=34,
                              #cutLeft=73,
                              #cutRight=87,#######硅胶
                          
                              #cutTop=44,
                              #cutBottom=44,
                              #cutLeft=6,
                              #cutRight=154,#####载体
                          
                              cutTop=54,
                              cutBottom=34,
                              cutLeft=42,
                              cutRight=118,#####载体
                          
                          
                              cPointRow=5,
                              cPointCol=5
                              )
    #tps_base0 = np.loadtxt(r'A_val_real33.txt').astype(np.float32)
    #tps_base0 = np.loadtxt(r'A_val_real.txt').astype(np.float32)
    tps_base0 = np.loadtxt(r'A_val_real55.txt').astype(np.float32)
    #tps_base0 = np.load(r'./groundtruth_z//tps_trained.npy').astype(np.float32)
    #ids = range(1950,2000)
    
    #feature_in = 46.4 * np.ones([params.data_size, params.cPointRow * params.cPointCol, 1], dtype=np.float32)###硅胶
    feature_in = 83.2 * np.ones([params.data_size, params.cPointRow * params.cPointCol, 1], dtype=np.float32)###载体
    
    #feature_in = np.load('groundtruth_z/disp_batch_invivo_z.npy')[0:100]
    
    #feature_in = 83.2 * np.random.normal(size=(params.data_size, params.cPointRow * params.cPointCol, 1))
    
    #feature_in = np.load('groundtruth_z/'+'feature_trained_final'+'.npy')[0:50,:,:]
    #feature_in0 = np.load('groundtruth_z/feature_test.npy')
    #feature_in0 = np.load('groundtruth_z/'+'feature_trained_final'+'.npy')
    #print(feature_in0.shape)

    #source_img_path = '/home/ubuntu/jupyter_workspace/dataset/guijiaoheart/phantom1_rect/'####硅胶
    source_img_path = '/home/ubuntu/jupyter_workspace/dataset/real_heart/invivo1_rect/'######载体
    
    #left_ims, right_ims = read_stereo_images(source_img_path, ids)
    #train_z(params, feature_in, tps_base0, left_ims, right_ims, 'groundtruth_z/',1)
    
    loss = []
    disps = []
    for i in range(200,300):
        ids  = range(i*params.data_size,(i+1)*params.data_size)
        left_ims, right_ims = read_stereo_images(source_img_path, ids)
        left_ims = np.array(left_ims, dtype = np.float32)
        right_ims = np.array(right_ims, dtype = np.float32)
        print(left_ims.shape)
        #feature_in = feature_in0[i*params.data_size:(i+1)*params.data_size,:,:]
        #train_z(params, feature_in, tps_base0, left_ims, right_ims, 'groundtruth_z/',i) 
        #right_ims = np.array(right_ims)
        #left_ims = np.array(left_ims)
        #print(right_ims.shape)
        #for n in range(params.data_size):
        #    for ii in range(50):
         #       for jj in range(50):
         #           #right_ims[n,130+ii,160+jj,:] = [0,0,0]
         #           left_ims[n,130+ii,160+jj,:] = [0,0,0]
        #print(right_ims.shape)
        #new_feature, res_loss, disp = train_z(params, feature_in, tps_base0, left_ims, right_ims, 'output/guijiao/',i)####硅胶
        new_feature, res_loss, disp = train_z(params, feature_in, tps_base0, left_ims, right_ims, 'groundtruth_z/',i)####载体
        loss.append(res_loss)
        disps.append(disp)
        feature_in = new_feature
    #loss = np.array(loss)
    #print(loss.shape)
    #np.save('output/guijiao/'+ 'TPSloss_per_img_base44.npy', loss)#####硅胶
    np.save('output/'+ 'TPSloss_per_img_base55.npy', loss)#####载体
    #np.save('output/guijiao/'+ 'TPSdisp_per_img_base44.npy', disps)#####硅胶
    np.save('output/'+ 'TPSdisp_per_img_base55.npy', disps)#####硅胶

(1, 288, 360, 3)
Tensor("meshgrid/mul:0", shape=(288, 360), dtype=float32, device=/device:GPU:0) Tensor("meshgrid/mul_1:0", shape=(288, 360), dtype=float32, device=/device:GPU:0)
Tensor("Reshape:0", shape=(1, 103680), dtype=float32, device=/device:GPU:0) Tensor("Reshape_1:0", shape=(1, 103680), dtype=float32, device=/device:GPU:0)
Tensor("Tile:0", shape=(1, 103680), dtype=float32, device=/device:GPU:0) Tensor("Tile_1:0", shape=(1, 103680), dtype=float32, device=/device:GPU:0)
Tensor("Reshape_2:0", shape=(103680,), dtype=float32, device=/device:GPU:0) Tensor("Reshape_3:0", shape=(103680,), dtype=float32, device=/device:GPU:0)
--------------------FEATURE---------------------
Tensor("map/TensorArrayStack/TensorArrayGatherV3:0", shape=(1, 40000, 1), dtype=float32)
Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
1
1.1
1.2
2
140.44232158625672
亮度补偿 4.3
train_tps_Step：   0 | Reonstruction loss is 1

62.16253357323529
62.161664363260236
亮度补偿 4.3
train_tps_Step：  70 | Reonstruction loss is 62.16166305541992 | Norm loss is  0.0 | Total loss is 62.16166305541992  | feature_var_mean is0.0052371216006577015
62.16087383534974
62.16010950332421
62.15975318298353
62.15954460522313
62.159370790422805
62.15888845435189
62.1578412201799
62.15633772215704
62.15469082692391
62.15331334463129
亮度补偿 4.3
train_tps_Step：  80 | Reonstruction loss is 62.15331268310547 | Norm loss is  0.0 | Total loss is 62.15331268310547  | feature_var_mean is-0.019881591200828552
62.15253986876983
62.15249530663329
62.15284730913642
62.15336879432624
62.154506102013144
62.154623448419734
62.15455825597163
62.154175793609404
62.15246377567429
62.152063939085764
亮度补偿 4.3
train_tps_Step：  90 | Reonstruction loss is 62.15206527709961 | Norm loss is  0.0 | Total loss is 62.15206527709961  | feature_var_mean is0.009164733812212944
62.15185967474163
62.15162933409824
62.15143376185385
62.151461027673484
62.15150883048255
62

62.340181390426714
62.33976158616045
亮度补偿 4.3
train_tps_Step：  70 | Reonstruction loss is 62.339759826660156 | Norm loss is  0.0 | Total loss is 62.339759826660156  | feature_var_mean is0.014297179877758026
62.339044420538926
62.34226362284615
62.34225487623546
62.33895696131679
62.33916686344992
62.33918435529434
62.342429808449225
62.339972881949
62.34195749147205
62.34190063850258
亮度补偿 4.3
train_tps_Step：  80 | Reonstruction loss is 62.34189987182617 | Norm loss is  0.0 | Total loss is 62.34189987182617  | feature_var_mean is-0.0045217894949018955
62.33854590297274
62.3386464810782
62.342158663517885
62.34211493046445
62.338510919283884
62.33837098452847
62.33835786564514
62.33838847637289
62.34187877197586
62.34193125163999
亮度补偿 4.3
train_tps_Step：  90 | Reonstruction loss is 62.3419303894043 | Norm loss is  0.0 | Total loss is 62.3419303894043  | feature_var_mean is0.0006814575172029436
62.341896265197235
62.34184815883845
62.34181754570104
62.34177818595294
62.34176943934226
62.3

60.508770549830885
60.50845591127192
60.5083422917923
60.5083728816522
60.50835977171224
60.50828111207251
60.50825926217258
亮度补偿 4.3
train_tps_Step：  70 | Reonstruction loss is 60.50825881958008 | Norm loss is  0.0 | Total loss is 60.50825881958008  | feature_var_mean is0.004305114969611168
60.508193712472796
60.50824615223262
60.50826800213255
60.508176232552856
60.50803639319332
60.50812379279303
60.50816749259288
60.50808883295314
60.508062613073236
60.508176232552856
亮度补偿 4.3
train_tps_Step：  80 | Reonstruction loss is 60.508174896240234 | Norm loss is  0.0 | Total loss is 60.508174896240234  | feature_var_mean is-0.0006668090936727822
60.50803639319332
60.508075723013185
60.50809320293313
60.508062613073236
60.508058243093245
60.508058243093245
60.508018913273375
60.508018913273375
60.508001433353435
60.50799269339346
亮度补偿 4.3
train_tps_Step：  90 | Reonstruction loss is 60.507991790771484 | Norm loss is  0.0 | Total loss is 60.507991790771484  | feature_var_mean is0.0002572631929

68.50316880273982
68.50252445009356
68.4641473124829
68.46281436755736
68.45817003848462
68.43381794749573
68.41006524176532
68.41003875661025
68.40942278489953
68.4536242629665
亮度补偿 4.3
train_tps_Step：  50 | Reonstruction loss is 68.45362091064453 | Norm loss is  0.0 | Total loss is 68.45362091064453  | feature_var_mean is0.0074038696475327015
68.45307259118032
68.45390671892102
68.4546658192988
68.45421565512127
68.4531608586661
68.45263566712566
68.45300639056597
68.42908092551886
68.45271510786287
68.45181036613353
亮度补偿 4.3
train_tps_Step：  60 | Reonstruction loss is 68.45181274414062 | Norm loss is  0.0 | Total loss is 68.45181274414062  | feature_var_mean is-0.0056243897415697575
68.4514881898104
68.45181477950781
68.45210164883663
68.45177947251351
68.45117042686157
68.45110863962151
68.45147053631324
68.45162500441337
68.45132489496169
68.42642372237965
亮度补偿 4.3
train_tps_Step：  70 | Reonstruction loss is 68.42642211914062 | Norm loss is  0.0 | Total loss is 68.42642211914062  

...

Tensor("map/TensorArrayStack/TensorArrayGatherV3:0", shape=(1, 40000, 1), dtype=float32)
1
1.1
1.2
2
71.75716608533739
亮度补偿 4.3
train_tps_Step：   0 | Reonstruction loss is 71.75716400146484 | Norm loss is  0.0 | Total loss is 71.75716400146484  | feature_var_mean is 0.0
67.70709162232765
65.94964490683718
65.47378875588268
65.47400295162994
65.77703529000699
65.97610748573875
65.87152805081561
65.60865310687076
65.32075273890024
65.11872876099623
亮度补偿 4.3
train_tps_Step：  10 | Reonstruction loss is 65.11872863769531 | Norm loss is  0.0 | Total loss is 65.11872863769531  | feature_var_mean is-0.7082278728485107
64.95267673061467
64.87895228859242
64.86489945315094
64.85153349581138
64.83297513585295
64.86552052484909
64.90766462253437
64.92321366177498
64.89287929212198
64.81846212974853
亮度补偿 4.3
train_tps_Step：  20 | Reonstruction loss is 64.8184585571289 | Norm loss is  0.0 | Total loss is 64.8184585571289  | feature_var_mean is0.12689299881458282
64.73099499489007
64.65558176100629
6

Tensor("map/TensorArrayStack/TensorArrayGatherV3:0", shape=(1, 40000, 1), dtype=float32)
1
1.1
1.2
2
74.67219447130017
亮度补偿 4.3
train_tps_Step：   0 | Reonstruction loss is 74.67219543457031 | Norm loss is  0.0 | Total loss is 74.67219543457031  | feature_var_mean is 0.0
75.0287212809566
74.34557486858483
74.00643006188263
73.9881563275212
74.04072412372611
73.79581205769058
73.65728696337705
73.74573189301518
73.85313686510494
73.8550185374241
亮度补偿 4.3
train_tps_Step：  10 | Reonstruction loss is 73.85501861572266 | Norm loss is  0.0 | Total loss is 73.85501861572266  | feature_var_mean is0.17844818532466888
73.79655051983988
73.7420477137177
73.66857449952975
73.56806779903964
73.58123012139946
73.57951247043647
73.57504165994732
73.57739135108984
73.60026157139914
73.68294911064538
亮度补偿 4.3
train_tps_Step：  20 | Reonstruction loss is 73.68295288085938 | Norm loss is  0.0 | Total loss is 73.68295288085938  | feature_var_mean is-0.0033242797944694757
73.6877742553687
73.69057789957282
7

Tensor("map/TensorArrayStack/TensorArrayGatherV3:0", shape=(1, 40000, 1), dtype=float32)
1
1.1
1.2
2
63.13296066363873
亮度补偿 4.3
train_tps_Step：   0 | Reonstruction loss is 63.13296127319336 | Norm loss is  0.0 | Total loss is 63.13296127319336  | feature_var_mean is 0.0
60.957298720842736
60.86214157706093
60.95357254849259
60.71959677419355
60.52037415331685
60.225585876237844
60.16390344395181
60.12104683343633
60.07321726046536
60.073946216049215
亮度补偿 4.3
train_tps_Step：  10 | Reonstruction loss is 60.07394790649414 | Norm loss is  0.0 | Total loss is 60.07394790649414  | feature_var_mean is0.4812176525592804
60.05800021491511
60.04429965968117
60.02638437357264
60.02764345583495
60.02578118005193
60.06831672530955
60.034835625268585
59.99501338418428
59.961615239322896
59.940300645519415
亮度补偿 4.3
train_tps_Step：  20 | Reonstruction loss is 59.94029998779297 | Norm loss is  0.0 | Total loss is 59.94029998779297  | feature_var_mean is0.08150085806846619
59.958014378698756
59.91304036

Tensor("map/TensorArrayStack/TensorArrayGatherV3:0", shape=(1, 40000, 1), dtype=float32)
1
1.1
1.2
2
66.49305387612522
亮度补偿 4.3
train_tps_Step：   0 | Reonstruction loss is 66.49305725097656 | Norm loss is  0.0 | Total loss is 66.49305725097656  | feature_var_mean is 0.0
60.748356706607204
57.906613287477384
56.707437468644734
56.60879517640524
56.85963898593568
57.09613954946482
57.24051086391889
57.23101591823207
57.02679315256241
56.68215533458748
亮度补偿 4.3
train_tps_Step：  10 | Reonstruction loss is 56.68215560913086 | Norm loss is  0.0 | Total loss is 56.68215560913086  | feature_var_mean is0.9377419948577881
56.23269535765168
55.907841925137525
55.747580905279
55.677779271407445
55.72317811951106
55.76562261952771
55.899532207764274
55.917988564463805
55.91477369463337
55.817162674409225
亮度补偿 4.3
train_tps_Step：  20 | Reonstruction loss is 55.817161560058594 | Norm loss is  0.0 | Total loss is 55.817161560058594  | feature_var_mean is-0.07854370027780533
55.70965400435519
55.626384

Tensor("map/TensorArrayStack/TensorArrayGatherV3:0", shape=(1, 40000, 1), dtype=float32)
1
1.1
1.2
2
66.0771895835515
亮度补偿 4.3
train_tps_Step：   0 | Reonstruction loss is 66.07718658447266 | Norm loss is  0.0 | Total loss is 66.07718658447266  | feature_var_mean is 0.0
62.4424520069808
60.25337310412797
59.47907017314318
59.394656928191836
59.65828492682331
59.73052401137335
59.71558416636137
59.63665305980043
59.570609937985715
59.396191522954666
亮度补偿 4.3
train_tps_Step：  10 | Reonstruction loss is 59.39619064331055 | Norm loss is  0.0 | Total loss is 59.39619064331055  | feature_var_mean is0.7400521636009216
59.217910812248704
59.06918716204431
58.95675312551801
58.911898155419436
58.83125714235866
58.84007258324827
58.939031517974755
58.985232288341095
59.06067266764357
59.061013578922584
亮度补偿 4.3
train_tps_Step：  20 | Reonstruction loss is 59.061012268066406 | Norm loss is  0.0 | Total loss is 59.061012268066406  | feature_var_mean is-0.19441132247447968
59.06698406054899
59.036839

In [5]:
import numpy as np
import scipy.misc
import imageio
import scipy.io as sio
import matplotlib.pyplot as plt
#import tensorflow as tf

disps = np.load('output/TPSdisp_per_img_base.npy',allow_pickle=True)####硅胶
#disps = np.load('output/guijiao/TPSdisp_per_img_base.npy',allow_pickle=True)####硅胶
disps = np.array(disps)
print(disps.shape)
disps = disps.reshape((100,288,360,1))
print(disps.shape)
np.save('output/'+ 'TPSdisp_per_img_base.npy', disps)###载体
#np.save('output/guijiao/'+ 'TPSdisp_per_img_base.npy', disps)###硅胶


loss = np.load('output/TPSloss_per_img_base.npy',allow_pickle=True)###载体
#loss = np.load('output/guijiao/TPSloss_per_img_base.npy',allow_pickle=True)###硅胶
loss = np.array(loss)
loss = loss.transpose((0,2,1))
print(loss.shape)
res_loss = []
for j in range(len(loss[0])):
    c = 0
    d = 0
    for i in range(len(loss)):
        c += loss[i][j][0]
        d += loss[i][j][1]
    res_loss.append(c/d)
res_loss = np.array(res_loss)
print(res_loss.shape)
print(res_loss)
np.save('output/'+ 'TPSloss_per_img_base.npy', res_loss)###载体
#np.save('output/guijiao/'+ 'TPSloss_per_img_base.npy', res_loss)###硅胶

(100, 1, 288, 360, 1)
(100, 288, 360, 1)
(100, 101, 2)
(101,)
[64.05217475 61.40491176 59.9974627  59.33487998 59.04600883 58.8156634
 58.63021514 58.467274   58.33600762 58.25675884 58.16648557 58.07571482
 58.01762918 57.95699002 57.9174254  57.87087479 57.82135223 57.78387779
 57.75949739 57.73812481 57.70475627 57.69010403 57.66670894 57.64695342
 57.62588539 57.61049756 57.59725303 57.58267342 57.5725725  57.57069587
 57.55920146 57.54493646 57.52946543 57.52232212 57.51013448 57.50313381
 57.49950024 57.49927383 57.49619738 57.49416504 57.49223078 57.49150044
 57.49185948 57.47581304 57.4754332  57.47411698 57.46921817 57.46982125
 57.46583252 57.46412797 57.46868007 57.46010148 57.45787496 57.45738603
 57.45770321 57.45529435 57.45701023 57.46315268 57.46103652 57.45601992
 57.45583764 57.4550234  57.45509629 57.453457   57.45260812 57.44916015
 57.45012715 57.45039879 57.44990922 57.44978213 57.45119455 57.45200751
 57.45100183 57.45090978 57.45192828 57.4487093  57.44858501 57

In [1]:
import numpy as np
import scipy.misc
import imageio
import scipy.io as sio
import matplotlib.pyplot as plt
#import tensorflow as tf


#loss = np.load('output/TPSloss_per_img_base.npy',allow_pickle=True)###载体
loss = np.load('output/guijiao/TPSloss_per_img_base44.npy',allow_pickle=True)###硅胶
loss = np.array(loss)
loss = loss.transpose((0,2,1))
print(loss.shape)
res_loss = []
for j in range(len(loss[0])):
    c = 0
    d = 0
    for i in range(len(loss)):
        c += loss[i][j][0]
        d += loss[i][j][1]
    res_loss.append(c/d)
res_loss = np.array(res_loss)
print(res_loss.shape)
print(res_loss)
#np.save('output/'+ 'TPSloss_per_img_base.npy', res_loss)###载体
np.save('output/guijiao/'+ 'TPSloss_per_img_base44.npy', res_loss)###硅胶

(100, 101, 2)
(101,)
[74.42910879 71.89145954 70.2327115  69.30639663 68.78427296 68.39229208
 68.07427883 67.85890425 67.71309458 67.58013487 67.44551408 67.32399846
 67.22718804 67.16738267 67.12665229 67.08468563 67.04355238 67.00442671
 66.96784317 66.93367879 66.89823883 66.85758117 66.81539512 66.77558138
 66.73970375 66.70687525 66.67494925 66.64522029 66.62107575 66.60364129
 66.59148513 66.58064458 66.57051667 66.56158558 66.55402346 66.54672383
 66.53867567 66.53031158 66.52286942 66.51652721 66.51068596 66.50470246
 66.49869692 66.49319629 66.48830625 66.48323946 66.47839871 66.47433217
 66.47157867 66.46967654 66.46820888 66.46694954 66.4658685  66.46473088
 66.46346958 66.46214571 66.46053563 66.45892725 66.45729467 66.45584129
 66.45470458 66.45383404 66.45308142 66.45242421 66.45183513 66.45124583
 66.45073029 66.45025608 66.44987029 66.44956646 66.44928942 66.449021
 66.4486455  66.44831133 66.44806829 66.44783662 66.44761208 66.44738575
 66.44712896 66.44698225 66.4468